# TensorFlow张量

TensorFlow如同名字所示，是一个定义和运行张量计算的框架。张量是向量和矩阵的泛化，可以表示更高维度。在内部，TensorFlow将张量表示为基础数据类型的n维数组。

当编写TensorFlow程序时，操作和传递的主要对象是`tf.Tensor`，它表示一个部分定义的计算，可以最终产生一个值。TensorFlow程序首先构建一个`tf.Tensor`计算图，描述如何基于其他张量来计算它，然后运行计算图相应部分来得到需要的结果。

`tf.Tensor`有下列属性： 

* 数据类型dtype(例如，float32, int32, 或者 string)
* 形状shape

张量中每个元素有相同的数据类型，数据类型通常是知道的。张量的形状（维度数和每个维度大小）可能是部分知道的。大多数操作如果完全知道输入的形状，那么可以产生形状完全知道的张量。但是，在某些情况下，只能在图执行时候才能确定张量的形状。

有些类型的张量比较特别，它们将在其他地方介绍，它们主要包括：

* tf.Variable
* tf.constant
* tf.placeholder
* tf.SparseTensor

只有`tf.Variable`张量的值是可变的，这意味着在一次简单执行上下文中，张量只有一个值，但是，执行同一个张量两次，可能会返回不同的值。例如，从磁盘的读取数据的张量和产生随机数的张量等。

In [ ]:
import tensorflow as tf

## 阶Rank

`tf.Tensor`的阶是张量维度数，它不同于数学里面所讨论的矩阵的Rank。不同阶的张量对应数学里不同的概念：
<table>
    <tr><td>阶</td><td>数学对象</td></tr>
    <tr><td>0</td><td>标量</td></tr>
    <tr><td>1</td><td>向量</td></tr>
    <tr><td>2</td><td>矩阵</td></tr>
    <tr><td>3</td><td>立方体</td></tr>
    <tr><td>n</td><td>只能想象</td></tr>
</table>

### 0阶

下列代码展示如何创建0阶变量或标量。第一个参数是值，第二个参数是类型。

In [ ]:
mammal = tf.Variable("Elephant", tf.string)
ignition = tf.Variable(451, tf.int16)
floating = tf.Variable(3.14159265359, tf.float64)
its_complicated = tf.Variable(12.3 - 4.85j, tf.complex64)

注意：在TensorFlow中，字符串作为单个对象，而不是字符的序列。可以有标量字符串、字符串向量等。

### 1阶

为了创建1阶张量，需要将一个列表作为初始值，例如：

In [ ]:
mystr = tf.Variable(["Hello"], tf.string)
cool_numbers  = tf.Variable([3.14159, 2.71828], tf.float32)
first_primes = tf.Variable([2, 3, 5, 7, 11], tf.int32)
its_very_complicated = tf.Variable([12.3 - 4.85j, 7.5 - 6.23j], tf.complex64)

### 高阶

2阶张量至少包括一行一列：

In [ ]:
mymat = tf.Variable([[7],[11]], tf.int16)
myxor = tf.Variable([[False, True],[True, False]], tf.bool)
linear_squares = tf.Variable([[4], [9], [16], [25]], tf.int32)
squarish_squares = tf.Variable([ [4, 9], [16, 25] ], tf.int32)
rank_of_squares = tf.rank(squarish_squares)
mymatC = tf.Variable([[7],[11]], tf.int32)

高阶张量包括多维数组，例如，在图像处理中使用4阶张量，维度分别对应批次、图像高度、图像宽度和颜色信道。

In [ ]:
my_image = tf.zeros([10, 299, 299, 3])  # 批 x 高度 x 宽度 x 颜色

### 获得张量的阶

为了确定张量的阶，调用`tf.rank`方法，例如，下列代码获得前面定义的张量的阶：

In [ ]:
r = tf.rank(my_image)
# 图运行后，r的值为4
print(r)
print(r.numpy())

`tf.rank`返回的结果也是一个张量。

### 张量切片

因为张量是一个n维数组，为了访问一个张量中单元格，需要指定n个索引。

对于0阶张量（标量），不需要提供索引，因为它已经是一个值。

对于1阶张量（向量），传递一个索引可以访问张量中的一个数：

In [ ]:
my_scalar = first_primes[2]
print(my_scalar)

注意：在中括号内的索引本身可以是一个标量张量，如果你想动态从向量中选择一个元素。

对于2阶或更高阶张量，情况更加有趣。对于一个2阶张量，传递2个索引，返回一个张量：

In [ ]:
my_scalar = squarish_squares[0, 1]
print(my_scalar)

但是，传递1个索引，将返回矩阵的子向量：

In [ ]:
my_row_vector = squarish_squares[1]
print(my_row_vector)
my_column_vector = squarish_squares[:, 1]
print(my_column_vector)

记号`:`是python的切片语法， 表示不管这一维。这在高阶张量中有用，它允许访问子向量、子矩阵甚至子张量。

## 形状Shape

张量的形状是每个维度的元素数量。TensorFlow自动从图构建过程中推导张量形状。推导的形状可能有或没有知道的阶，如果阶知道，每个维度大小也可能不知道。

TensorFlow使用3种记号习惯描述张量的维度信息：阶、形状和维度数。下表解释它们间的关系：

<table>
    <tr><td>阶</td><td>形状</td><td>维数</td></tr>
    <tr><td>0</td><td>[]</td><td>0维</td></tr>
    <tr><td>1</td><td>[D0]</td><td>1维</td></tr>
    <tr><td>2</td><td>[D0, D1]</td><td>2维</td></tr>
    <tr><td>3</td><td>[D0, D1, D2]</td><td>3维</td></tr>
    <tr><td>n</td><td>[D0, D1, ... Dn-1]</td><td>n维</td></tr>
</table>


形状通过python整数列表或元组或者`tf.TensorShape`表示。

### 获得张量的形状

有两种方式访问张量的形状。当构建计算图时，询问关于张量形状的信息是有用的。这可以通过读取一个张量对象的`shape`属性实现，这个方法返回`TensorShape`对象，它可以方便的表示部分确定的形状（因为构建图时，不是所有形状都知道）。

可以获得一个表示其他张量运行时完全定义形状的张量。这通过调用`tf.shape`操作实现。使用这种方法，你可以构建操作张量形状的计算图，图中张量依赖输入张量的动态形状。

例如，这里创建了一个零张量，形状和一个给定矩阵的列大小相同：

In [ ]:
zeros = tf.zeros(mymat.shape[1])
print(zeros.shape)
print(tf.shape(zeros))

### 改变张量的形状

张量中元素的个数是形状的乘积，标量的元素个数为1。由于不同形状的张量有相同数量的元素，可以方便的改变张量的形状，而保持元素数量不变。这可以通过`tf.reshape`实现。

下列代码展示如何改变张量形状：

In [ ]:
rank_three_tensor = tf.ones([3, 4, 5])

# 将已有内容转换成6*10的2维矩阵
matrix = tf.reshape(rank_three_tensor, [6, 10])  

# 将已有内容转换成3*20的矩阵，-1表示让reshape计算该维大小
matrixB = tf.reshape(matrix, [3, -1])  

# 将已有内容转换成4*3*5的张量
matrixAlt = tf.reshape(matrixB, [4, 3, -1])  

# 注意转换前后张量的元素数量不变。因此，下面的转换不行，因为最后一个维度
# 无法取值使得元素数量保持不变。
# yet_another = tf.reshape(matrixAlt, [13, 2, -1])  # ERROR!

## 数据类型

除了维度外，张量还有数据类型。

一个张量不可能有多个数据类型。但是，可以将任意数据结构序列化为字符串，并保存在张量中。

使用`tf.cast`可以将张量从一种数据类型转换成另外一种数据类型。

In [ ]:
# 将整型张量转换成浮点型张量
float_tensor = tf.cast(tf.constant([1, 2, 3]), dtype=tf.float32)
print(float_tensor)
print(float_tensor.dtype)

为了查看张量的数据类型，使用`dtype`属性。

当从python对象创建张量时，你可能需要指定数据类型。如果没有指定，TensorFlow选择能表示数据的类型。TensorFlow将python整数转换成`tf.int32`,将python实数转换成`tf.float32`。TensorFlow使用numpy使用的规则转换数组。

## 张量计算

`以下内容对TensorFlow2不适应。TF2采用即刻执行，不再需要显式调用张量计算方法eval。`

一旦计算图构建好，你可以运行产生`tf.Tensor`的计算图，获得赋给张量的值。这对于调试和很多TensorFlow工作的地方都有用。

计算张量最简单的方法是使用`Tensor.eval` 方法，例如:

In [ ]:
print(tf.executing_eagerly())
constant = tf.constant([1, 2, 3])
tensor = constant * constant  # 向量对应元素相乘

# print(tensor.eval())
print(tensor)

`eval`方法只有缺省的`tf.Session`激活时才可用。

`Tensor.eval`返回一个和张量有相同内容的numpy数组。

有时，张量计算所依赖的动态信息不可用，因而缺乏上下文，计算张量也不可能了。例如，依赖占位符的张量在没有提供占位符的值的前提下不能计算。

In [ ]:
# p = tf.placeholder(tf.float32)
# t = p + 1.0
# t.eval()  # 这将失败，因为占位符没有值
# t.eval(feed_dict={p:2.0})  # 这将成功，因为我们给占位符喂了一个值

注意，可以feed任何张量，而不仅仅是占位符。

其他模型结构可能使得张量计算变得复杂。TensorFlow不能直接计算函数或控制流内定义的张量。如果张量依赖队列中的值，只有某些东西入列才能计算张量，否则，计算将挂起。当和队列一起工作时，记得在计算任何张量前调用`tf.train.start_queue_runners`